In [1]:
# test.py
from core.prepare import prepare_documents, build_index
from core.query import query_answer, init_chat_engine, chat_turn
from core.query import suggest_recipes_by_ingredients, find_similar_recipes
from core.utils import get_keywords_from_llama, scale_ingredients
from core.embedding import get_embed_model
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
import json
import nest_asyncio

In [ ]:
nest_asyncio.apply()

with open("sample.json", 'r', encoding='utf-8') as f:
    recipe_data = json.load(f)

In [ ]:
Settings.llm = Ollama(model="tinyllama:1.1b", request_timeout=600.0)
Settings.embed_model = get_embed_model("nomic-embed-text")
Settings.chunk_size = 256

# 准备文档和索引
documents = prepare_documents(recipe_data)
index = build_index(documents)

100%|██████████| 20/20 [00:11<00:00,  1.70it/s]


[INFO] Ingestion 完成，生成 20 个节点


In [11]:
# Ingredient Scaling Test
print("="*20, "Ingredient Scaling", "="*20)
ingredients = [
    "2 tablespoons olive oil",
    "3 chicken breasts",
    "1/2 cup lemon juice"
]
scaled = scale_ingredients(ingredients, scale_by=2)
print("Original Ingredients:", ingredients)
print("Scaled Ingredients:", scaled)

==================== Ingredient Scaling ====================
Original Ingredients: ['2 tablespoons olive oil', '3 chicken breasts', '1/2 cup lemon juice']
Scaled Ingredients: ['4.0 tablespoons olive oil', '6.0 chicken breasts', '1.0 cup lemon juice']


In [5]:
print("="*20, "Test: Single Question Answer", "="*20)
question = "What are the ingredients for chicken?"
answer = query_answer(question, index)
print(f"Q: {question}\nA: {answer}\n")

==================== Test: Single Question Answer ====================
Q: What are the ingredients for chicken?
A: Yes, the given context information includes the list of ingredients for the Spicy and Savory ChiCKEN Dips and Garlicky Breast Delights recipe with a picture of the ingredients. The instructions include baking the chiCKen breast in a baaking sheet, cooking it with other ingredients including onion, garlic, BBQ sauce, white wine sauce, and more, while being seasoned with salt, black pepper, smoked paprika, and paprikas. The given document includes ingredient excerpts from the recipe, such as "chiCKen Breast", "Zuccini", "Yellaw SquaSh", "Celery", "Basil", "ParslEy", "Thyme", "PaprikA", "Salt", and "Pepper". Additionally, the given context information includes a recipe for Spicy and Savory ChiCKEN Dips and Garlicky Breast Delights, which consists of a list of ingredients including ChiCKen breast, onion, garlic, BBQ sauce, white wine sauce, smoked paprika, and more.



In [6]:
print("="*20, "Test: Multi-turn Chat", "="*20)
chat_engine = init_chat_engine(index)
query1 = "Tell me how to cook chicken."
response1 = chat_turn(query1, chat_engine)
print(f"Q1: {query1}\nA1: {response1}\n")

query2 = "Any tips for making it tastier?"
response2 = chat_turn(query2, chat_engine)
print(f"Q2: {query2}\nA2: {response2}\n")

==================== Test: Multi-turn Chat ====================
Q1: Tell me how to cook chicken.
A1: Sure! Here's how to cook Chicken:

Ingredients:
- Chicken breast (or chicken thigh)
- Salt and black pepper, to taste
- Olive oil, for pan-frying

Instructions:
1. Preheat your oven to 400°F (200°C). Line a baking sheet with parchment paper or grease it well with cooking spray.
2. In a small bowl, mix together the salt and black pepper.
3. Heat some olive oil in a large skillet over medium-high heat until hot but not smoking.
4. Place the Chicken breast in the skillet and sear it on all sides, about 1-2 minutes per side, until browned and crispy.
5. Transfer the Chicken to the prepared baking sheet.
6. Drizzle some olive oil over the top of the Chicken, season with salt and black pepper, and sprinkle with paprika or garlic powder if desired.
7. Bake for 15-20 minutes until the internal temperature reaches 165°F (74°C) for medium-rare, or 180°F (85°C) for medium.
8. Allow the Chicken to 

In [7]:
print("="*20, "Test: Suggest Recipes Based on Ingredients", "="*20)
ingredients = ["chicken", "garlic", "lemon"]
suggestions = suggest_recipes_by_ingredients(ingredients, index)
print(f"Available Ingredients: {ingredients}")
print(f"Suggested Recipes: {suggestions}\n")

==================== Test: Suggest Recipes Based on Ingredients ====================
Available Ingredients: ['chicken', 'garlic', 'lemon']
Suggested Recipes: [('chicken', 2), ('olive oil', 2), ('rice', 2)]



In [9]:
print("="*20, "Test: Find Similar Recipes", "="*20)
similar_recipes = find_similar_recipes("chicken", index, Settings.embed_model)
print(f"Recipes similar to 'drumsticks': {similar_recipes}\n")

==================== Test: Find Similar Recipes ====================
Recipes similar to 'drumsticks': [('olive oil', 0.8730856596257394), ('salt', 0.8547947415314653), ('sugar', 0.846239825364511)]



In [10]:
print("="*20, "Test: Extract Keywords", "="*20)
query_text = "I would like to cook something with garlic and lemon."
keywords = get_keywords_from_llama(query_text)
print(f"Input Text: {query_text}")
print(f"Extracted Keywords: {keywords}")

==================== Test: Extract Keywords ====================
[WARN] LLM output does not contain 'Keywords:' prefix. Raw output: I am unable to extract keywords from the provided user input as it does not follow proper english syntax. However, here is a sample response based on a similar request:

keywords: garlic, lemon, cooking

please note that each keyword should be separated by a comma and in this format only. The above example includes only three keywords and adheres to the recommended 3-word limit for effective keyword extraction.
Input Text: I would like to cook something with garlic and lemon.
Extracted Keywords: []
